In [6]:
import numpy as np
import PIL.Image

import torch, torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from matplotlib import pyplot as plt

import os


numero_de_epocas = 20
bs = 32
dataset = r'./data/Fer-2013/'
pasta_treino = os.path.join(dataset, 'treino')
pasta_validacao = os.path.join(dataset, 'validacao')
pasta_teste = os.path.join(dataset, 'teste')
numero_de_classes = len(os.listdir(pasta_validacao))

from torchvision import transforms
image_size = 48
transformacoes_de_imagens = { 
    'treino': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.Resize(size=[image_size, image_size]),
        transforms.RandomRotation(20),
        transforms.Grayscale(num_output_channels=3), 
        transforms.ToTensor()
    ]),

    'validacao': transforms.Compose([
        transforms.Resize(size=[image_size, image_size]),
        transforms.Grayscale(num_output_channels=3),  
        transforms.ToTensor(),
    ])
}


data = {
    'treino': datasets.ImageFolder(root=pasta_treino, transform=transformacoes_de_imagens['treino']),
    'validacao': datasets.ImageFolder(root=pasta_validacao, transform=transformacoes_de_imagens['validacao'])
}

# Mapear os índices com os nomes das classes
indice_para_classe = {v: k for k, v in data['treino'].class_to_idx.items()}
print(indice_para_classe)

# Quantidade de imagens para serem utilizadas para calcular erro médio e acurácia
num_imagens_treino = len(data['treino'])
num_imagens_validacao = len(data['validacao'])

# Cria os DataLoaders para treino e validação
data_loader_treino = DataLoader(data['treino'], batch_size=bs, shuffle=True)
data_loader_validacao = DataLoader(data['validacao'], batch_size=bs, shuffle=True)

# Carregar o modelo ResNet-50 pré-treinado
resnet50 = models.resnet50(pretrained=True)

# Congelar os parâmetros da rede pré-treinada
for param in list(resnet50.parameters())[-4:]:
    param.requires_grad = True

# Modificar a última camada totalmente conectada (fc) para o número de classes do problema
num_features = resnet50.fc.in_features
resnet50.fc = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, numero_de_classes),
    nn.LogSoftmax(dim=1)
)

# Definir algoritmo de otimização e função de erro
funcao_erro = nn.NLLLoss()  # Negative Log Likelihood Loss
otimizador = optim.Adam(resnet50.parameters(), lr=0.0001)

# Função para treinamento e validação
from tqdm import tqdm

def treinar_e_validar(modelo, metrica_erro, otimizador_sgd, epocas=25):
    historico = []
    melhor_acuracia = 0.0
    melhor_modelo = None

    for epoca in range(epocas):
        inicio_epoca = time.time()
        print("\n\nÉpoca: {}/{}".format(epoca + 1, epocas))

        modelo.train()
        erro_treino = 0.0
        acuracia_treino = 0.0
        erro_validacao = 0.0
        acuracia_validacao = 0.0

        for i, (entradas, labels) in enumerate(tqdm(data_loader_treino, desc="Treinando")):
            entradas = entradas.to(device)
            labels = labels.to(device)

            otimizador_sgd.zero_grad()
            saidas = modelo(entradas)
            erro = metrica_erro(saidas, labels)
            erro.backward()
            otimizador_sgd.step()

            erro_treino += erro.item() * entradas.size(0)
            valores_maximos, indices_dos_valores_maximos = torch.max(saidas.data, 1)
            predicoes_corretas = indices_dos_valores_maximos.eq(labels.data.view_as(indices_dos_valores_maximos))
            acuracia = torch.mean(predicoes_corretas.type(torch.FloatTensor))
            acuracia_treino += acuracia.item() * entradas.size(0)

        # Dicionários para armazenar acertos e totais por classe
        acertos_por_classe = {classe: 0 for classe in indice_para_classe.values()}
        totais_por_classe = {classe: 0 for classe in indice_para_classe.values()}

        with torch.no_grad():
            modelo.eval()
            for j, (entradas, labels) in enumerate(tqdm(data_loader_validacao, desc="Validando")):
                entradas = entradas.to(device)
                labels = labels.to(device)
                saidas = modelo(entradas)
                erro = metrica_erro(saidas, labels)
                erro_validacao += erro.item() * entradas.size(0)

                valores_maximos, indices_dos_valores_maximos = torch.max(saidas.data, 1)
                predicoes_corretas = indices_dos_valores_maximos.eq(labels.data.view_as(indices_dos_valores_maximos))
                acuracia = torch.mean(predicoes_corretas.type(torch.FloatTensor))
                acuracia_validacao += acuracia.item() * entradas.size(0)

                # Atualizar contadores por classe
                for label, predicao in zip(labels, indices_dos_valores_maximos):
                    classe_real = indice_para_classe[label.item()]
                    totais_por_classe[classe_real] += 1
                    if label == predicao:
                        acertos_por_classe[classe_real] += 1

        erro_medio_treino = erro_treino / num_imagens_treino
        acuracia_media_treino = acuracia_treino / num_imagens_treino
        erro_medio_validacao = erro_validacao / num_imagens_validacao
        acuracia_media_validacao = acuracia_validacao / num_imagens_validacao

        historico.append([erro_medio_treino, erro_medio_validacao, acuracia_media_treino, acuracia_media_validacao])

        fim_epoca = time.time()

        print("Época : {:03d}, Treino: Erro: {:.4f}, Acurácia: {:.4f}%, \n\t\tValidação : Erro : {:.4f}, Acurácia: {:.4f}%, Tempo: {:.4f}s".format(
            epoca + 1, erro_medio_treino, acuracia_media_treino * 100, erro_medio_validacao, acuracia_media_validacao * 100, fim_epoca - inicio_epoca))



        print("\nAcurácia por classe na validação:")
        for classe, total in totais_por_classe.items():
            if total > 0:
                acuracia_classe = (acertos_por_classe[classe] / total) * 100
                print(f"Classe '{classe}': {acuracia_classe:.2f}%")
            else:
                print(f"Classe '{classe}': Sem exemplos na validação")

        if acuracia_media_validacao > melhor_acuracia:
            melhor_acuracia = acuracia_media_validacao
            torch.save(modelo, './melhores_modelos/melhor_modelo01k.pt')
            melhor_modelo = modelo

    return melhor_modelo, historico


# Se você for rico e tiver uma RTX 3080 (que sonho!!)
# Mas se não for, sem problemas, sua CPU provavelmente vai dar conta do recado!
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Imprime se vai ser CPU ou alguma GPU
print(f'{device}\n')

# Define o número de épocas
numero_de_epocas = 20

# treina o modelo! Finalmente!!
resnet50 = resnet50.to(device)
# modelo_treinado, historico = treinar_e_validar(resnet50, funcao_erro, otimizador, numero_de_epocas)


{0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Neutral', 5: 'Sad', 6: 'Surprise'}
cuda:0



In [ ]:
import numpy as np
import PIL.Image

import torch, torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from matplotlib import pyplot as plt

import os


image_size = 48
numero_de_epocas = 80
bs = 32
dataset = r'./data/Fer-2013/'
pasta_treino = os.path.join(dataset, 'treino')
pasta_validacao = os.path.join(dataset, 'validacao')
pasta_teste = os.path.join(dataset, 'teste')
numero_de_classes = len(os.listdir(pasta_validacao))

from tqdm import tqdm

# Atualização das transformações para melhorar a generalização
definicoes_de_transformacoes = {
    'treino': transforms.Compose([
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.2),
        transforms.RandomRotation(degrees=30),
        transforms.RandomResizedCrop(size=image_size, scale=(0.8, 1.0)),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),

    'validacao': transforms.Compose([
        transforms.Resize(size=(image_size, image_size)),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

# Aplicação das transformações atualizadas no carregamento dos dados
transformacoes_de_imagens = definicoes_de_transformacoes


data = {
    'treino': datasets.ImageFolder(root=pasta_treino, transform=transformacoes_de_imagens['treino']),
    'validacao': datasets.ImageFolder(root=pasta_validacao, transform=transformacoes_de_imagens['validacao'])
}

# Mapear os índices com os nomes das classes
indice_para_classe = {v: k for k, v in data['treino'].class_to_idx.items()}
print(indice_para_classe)

# Quantidade de imagens para serem utilizadas para calcular erro médio e acurácia
num_imagens_treino = len(data['treino'])
num_imagens_validacao = len(data['validacao'])

# Cria os DataLoaders para treino e validação
data_loader_treino = DataLoader(data['treino'], batch_size=bs, shuffle=True)
data_loader_validacao = DataLoader(data['validacao'], batch_size=bs, shuffle=True)

# Carregar o modelo ResNet-50 pré-treinado
resnet50 = models.resnet50(pretrained=True)

# Descongelar mais camadas do modelo pré-treinado
for param in list(resnet50.parameters())[:-10]:
    param.requires_grad = True

# Modificar a última camada totalmente conectada (fc) para o número de classes do problema
num_features = resnet50.fc.in_features
resnet50.fc = nn.Sequential(
    nn.Linear(num_features, 1024),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(512, numero_de_classes),
    nn.LogSoftmax(dim=1)
)

# Definir algoritmo de otimização e função de erro
funcao_erro = nn.NLLLoss()  # Negative Log Likelihood Loss
otimizador = optim.Adam(resnet50.parameters(), lr=0.0001)

# Função principal de treinamento e validação
def treinar_e_validar_com_checkpoint(modelo, metrica_erro, otimizador_sgd, epocas=25, epoca_inicial=0):
    historico = []
    melhor_acuracia = 0.0
    melhor_modelo = None

    for epoca in range(epoca_inicial, epocas):
        inicio_epoca = time.time()
        print(f"\n\nÉpoca: {epoca + 1}/{epocas}")

        modelo.train()
        erro_treino = 0.0
        acuracia_treino = 0.0
        erro_validacao = 0.0
        acuracia_validacao = 0.0

        for i, (entradas, labels) in enumerate(tqdm(data_loader_treino, desc="Treinando")):
            entradas = entradas.to(device)
            labels = labels.to(device)

            otimizador_sgd.zero_grad()
            saidas = modelo(entradas)
            erro = metrica_erro(saidas, labels)
            erro.backward()
            otimizador_sgd.step()

            erro_treino += erro.item() * entradas.size(0)
            valores_maximos, indices_dos_valores_maximos = torch.max(saidas.data, 1)
            predicoes_corretas = indices_dos_valores_maximos.eq(labels.data.view_as(indices_dos_valores_maximos))
            acuracia = torch.mean(predicoes_corretas.type(torch.FloatTensor))
            acuracia_treino += acuracia.item() * entradas.size(0)

        with torch.no_grad():
            modelo.eval()
            for j, (entradas, labels) in enumerate(tqdm(data_loader_validacao, desc="Validando")):
                entradas = entradas.to(device)
                labels = labels.to(device)
                saidas = modelo(entradas)
                erro = metrica_erro(saidas, labels)
                erro_validacao += erro.item() * entradas.size(0)

                valores_maximos, indices_dos_valores_maximos = torch.max(saidas.data, 1)
                predicoes_corretas = indices_dos_valores_maximos.eq(labels.data.view_as(indices_dos_valores_maximos))
                acuracia = torch.mean(predicoes_corretas.type(torch.FloatTensor))
                acuracia_validacao += acuracia.item() * entradas.size(0)

        erro_medio_treino = erro_treino / num_imagens_treino
        acuracia_media_treino = acuracia_treino / num_imagens_treino
        erro_medio_validacao = erro_validacao / num_imagens_validacao
        acuracia_media_validacao = acuracia_validacao / num_imagens_validacao

        historico.append([erro_medio_treino, erro_medio_validacao, acuracia_media_treino, acuracia_media_validacao])

        fim_epoca = time.time()

        print(f"Época : {epoca + 1:03d}, Treino: Erro: {erro_medio_treino:.4f}, Acurácia: {acuracia_media_treino * 100:.4f}%, \n\t\tValidação : Erro : {erro_medio_validacao:.4f}, Acurácia: {acuracia_media_validacao * 100:.4f}%, Tempo: {fim_epoca - inicio_epoca:.4f}s")

        if acuracia_media_validacao > melhor_acuracia:
            melhor_acuracia = acuracia_media_validacao
            torch.save(modelo, './melhores_modelos/melhor_modelo001k.pt')
            melhor_modelo = modelo

        # Salvar o checkpoint ao final de cada época
        salvar_checkpoint(modelo, otimizador_sgd, epoca, caminho_checkpoint)

    return melhor_modelo, historico

import os
import torch

# Função para salvar o checkpoint
def salvar_checkpoint(modelo, otimizador, epoca, caminho):
    estado = {
        'epoca': epoca,
        'modelo_state_dict': modelo.state_dict(),
        'otimizador_state_dict': otimizador.state_dict()
    }
    torch.save(estado, caminho)

# Função para carregar o checkpoint
def carregar_checkpoint(caminho, modelo, otimizador):
    if os.path.exists(caminho):
        estado = torch.load(caminho)
        modelo.load_state_dict(estado['modelo_state_dict'])
        otimizador.load_state_dict(estado['otimizador_state_dict'])
        epoca_inicial = estado['epoca'] + 1
        print(f"Checkpoint carregado: retomando do treinamento da época {epoca_inicial}")
        return epoca_inicial
    else:
        print("Nenhum checkpoint encontrado. Iniciando treinamento do zero.")
        return 0

# Caminho do arquivo de checkpoint
caminho_checkpoint = './melhores_modelos/last_melhor001k.pt'

# Carregar o checkpoint se existir
epoca_inicial = carregar_checkpoint(caminho_checkpoint, resnet50, otimizador)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet50 = resnet50.to(device)
# Treina o modelo, retomando do checkpoint se necessário
# modelo_treinado, historico = treinar_e_validar_com_checkpoint(resnet50, funcao_erro, otimizador, numero_de_epocas, epoca_inicial)


{0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Neutral', 5: 'Sad', 6: 'Surprise'}


c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\Danil\AppData\Local\Temp\ipykernel_15184\678801155.py:177: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpick

Checkpoint carregado: retomando do treinamento da época 33


In [9]:
import numpy as np
import PIL.Image

import torch, torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from matplotlib import pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import KFold

import os


image_size = 48
numero_de_epocas = 20
bs = 32
dataset = r'./data/Fer-2013/'
pasta_treino = os.path.join(dataset, 'treino')
pasta_validacao = os.path.join(dataset, 'validacao')
pasta_teste = os.path.join(dataset, 'teste')
numero_de_classes = len(os.listdir(pasta_validacao))

from tqdm import tqdm

# Atualização das transformações para melhorar a generalização
definicoes_de_transformacoes = {
    'treino': transforms.Compose([
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.2),
        transforms.RandomRotation(degrees=30),
        transforms.RandomResizedCrop(size=image_size, scale=(0.8, 1.0)),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),

    'validacao': transforms.Compose([
        transforms.Resize(size=(image_size, image_size)),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

# Aplicação das transformações atualizadas no carregamento dos dados
transformacoes_de_imagens = definicoes_de_transformacoes


data = {
    'treino': datasets.ImageFolder(root=pasta_treino, transform=transformacoes_de_imagens['treino']),
    'validacao': datasets.ImageFolder(root=pasta_validacao, transform=transformacoes_de_imagens['validacao'])
}

# Mapear os índices com os nomes das classes
indice_para_classe = {v: k for k, v in data['treino'].class_to_idx.items()}
print(indice_para_classe)

# Quantidade de imagens para serem utilizadas para calcular erro médio e acurácia
num_imagens_treino = len(data['treino'])
num_imagens_validacao = len(data['validacao'])

# Calcular pesos para lidar com classes desbalanceadas
train_labels = [label for _, label in data['treino'].imgs]
class_weights = compute_class_weight('balanced', classes=np.arange(numero_de_classes), y=train_labels)
weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Criar os DataLoaders para treino e validação
data_loader_treino = DataLoader(data['treino'], batch_size=bs, shuffle=True)
data_loader_validacao = DataLoader(data['validacao'], batch_size=bs, shuffle=True)

# Carregar o modelo ResNet-50 pré-treinado
resnet50 = models.resnet50(pretrained=True)

# Descongelar mais camadas do modelo pré-treinado
for param in list(resnet50.parameters())[:-10]:
    param.requires_grad = True

# Modificar a última camada totalmente conectada (fc) para o número de classes do problema
num_features = resnet50.fc.in_features
resnet50.fc = nn.Sequential(
    nn.Linear(num_features, 1024),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(512, numero_de_classes),
    nn.LogSoftmax(dim=1)
)

# Definir algoritmo de otimização e função de erro
funcao_erro = nn.CrossEntropyLoss(weight=weights)
otimizador = optim.Adam(resnet50.parameters(), lr=0.0001)

# Função para salvar o checkpoint
def salvar_checkpoint(modelo, otimizador, epoca, caminho):
    estado = {
        'epoca': epoca,
        'modelo_state_dict': modelo.state_dict(),
        'otimizador_state_dict': otimizador.state_dict()
    }
    torch.save(estado, caminho)

# Função para carregar o checkpoint
def carregar_checkpoint(caminho, modelo, otimizador):
    if os.path.exists(caminho):
        estado = torch.load(caminho)
        modelo.load_state_dict(estado['modelo_state_dict'])
        otimizador.load_state_dict(estado['otimizador_state_dict'])
        epoca_inicial = estado['epoca'] + 1
        print(f"Checkpoint carregado: retomando do treinamento da época {epoca_inicial}")
        return epoca_inicial
    else:
        print("Nenhum checkpoint encontrado. Iniciando treinamento do zero.")
        return 0

# Função principal de treinamento e validação

def treinar_e_validar_cross_validation(modelo, metrica_erro, otimizador_sgd, epocas=25, folds=5):
    kf = KFold(n_splits=folds, shuffle=True, random_state=42)
    fold_results = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(data['treino'])):
        print(f"\nIniciando Fold {fold + 1}/{folds}")

        # Divisão dos dados por fold
        train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
        val_sampler = torch.utils.data.SubsetRandomSampler(val_idx)
        
        data_loader_train_fold = DataLoader(data['treino'], batch_size=bs, sampler=train_sampler)
        data_loader_val_fold = DataLoader(data['treino'], batch_size=bs, sampler=val_sampler)

        melhor_acuracia = 0.0
        melhor_modelo = None

        for epoca in range(epocas):
            inicio_epoca = time.time()
            print(f"\nFold {fold + 1}, Época {epoca + 1}/{epocas}")

            modelo.train()
            erro_treino = 0.0
            acuracia_treino = 0.0

            for entradas, labels in tqdm(data_loader_train_fold, desc="Treinando"):
                entradas = entradas.to(device)
                labels = labels.to(device)

                otimizador_sgd.zero_grad()
                saidas = modelo(entradas)
                erro = metrica_erro(saidas, labels)
                erro.backward()
                otimizador_sgd.step()

                erro_treino += erro.item() * entradas.size(0)

            erro_medio_treino = erro_treino / len(train_idx)

            modelo.eval()
            erro_validacao = 0.0
            acuracia_validacao = 0.0

            with torch.no_grad():
                for entradas, labels in tqdm(data_loader_val_fold, desc="Validando"):
                    entradas = entradas.to(device)
                    labels = labels.to(device)
                    saidas = modelo(entradas)
                    erro = metrica_erro(saidas, labels)
                    erro_validacao += erro.item() * entradas.size(0)

                erro_medio_validacao = erro_validacao / len(val_idx)

                print(f"\nFold {fold + 1}, Época {epoca + 1}: Erro Treino {erro_medio_treino:.4f}, Erro Validação {erro_medio_validacao:.4f}")
                # Salvar o melhor modelo baseado na validação
                if erro_medio_validacao < melhor_acuracia or melhor_modelo is None:
                    melhor_acuracia = erro_medio_validacao
                    melhor_modelo = modelo.state_dict()
                    torch.save(melhor_modelo, f'./melhores_modelos/melhor_modelo_a{fold}.pt')
                salvar_checkpoint(modelo, otimizador_sgd, epoca, caminho_checkpoint)


        fold_results.append(erro_medio_validacao)

    print(f"\nErro Médio Validação (Cross-Validation): {np.mean(fold_results):.4f}")

# Se você for rico e tiver uma RTX 3080 (que sonho!!)
# Mas se não for, sem problemas, sua CPU provavelmente vai dar conta do recado!
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Imprime se vai ser CPU ou alguma GPU
print(f'{device}\n')

# Define o número de épocas
numero_de_epocas = 80
caminho_checkpoint = './melhores_modelos/last_melhor_modelo_folda01.pt'

# Carregar o checkpoint se existir
epoca_inicial = carregar_checkpoint(caminho_checkpoint, resnet50, otimizador)


# # treina o modelo! Finalmente!!
# resnet50 = resnet50.to(device)
# treinar_e_validar_cross_validation(resnet50, funcao_erro, otimizador, numero_de_epocas, folds=5)


{0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Neutral', 5: 'Sad', 6: 'Surprise'}


c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


cuda:0



C:\Users\Danil\AppData\Local\Temp\ipykernel_19860\2332882134.py:114: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  estado = torch.load(caminho)


Checkpoint carregado: retomando do treinamento da época 72


In [10]:
import numpy as np
import PIL.Image

import torch, torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from matplotlib import pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import KFold

import os


image_size = 48
numero_de_epocas = 20
bs = 32
dataset = r'./data/Fer-2013/'
pasta_treino = os.path.join(dataset, 'treino')
pasta_validacao = os.path.join(dataset, 'validacao')
pasta_teste = os.path.join(dataset, 'teste')
numero_de_classes = len(os.listdir(pasta_validacao))

from tqdm import tqdm

definicoes_de_transformacoes = {
    'treino': transforms.Compose([
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.2),
        transforms.RandomRotation(degrees=30),
        transforms.RandomResizedCrop(size=image_size, scale=(0.8, 1.0)),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),

    'validacao': transforms.Compose([
        transforms.Resize(size=(image_size, image_size)),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

transformacoes_de_imagens = definicoes_de_transformacoes


data = {
    'treino': datasets.ImageFolder(root=pasta_treino, transform=transformacoes_de_imagens['treino']),
    'validacao': datasets.ImageFolder(root=pasta_validacao, transform=transformacoes_de_imagens['validacao'])
}

indice_para_classe = {v: k for k, v in data['treino'].class_to_idx.items()}
print(indice_para_classe)

num_imagens_treino = len(data['treino'])
num_imagens_validacao = len(data['validacao'])

train_labels = [label for _, label in data['treino'].imgs]
class_weights = compute_class_weight('balanced', classes=np.arange(numero_de_classes), y=train_labels)
weights = torch.tensor(class_weights, dtype=torch.float).to(device)

data_loader_treino = DataLoader(data['treino'], batch_size=bs, shuffle=True)
data_loader_validacao = DataLoader(data['validacao'], batch_size=bs, shuffle=True)

resnet50 = models.resnet50(pretrained=True)

for param in list(resnet50.parameters())[:-10]:
    param.requires_grad = True

num_features = resnet50.fc.in_features
resnet50.fc = nn.Sequential(
    nn.Linear(num_features, 1024),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(512, numero_de_classes),
    nn.LogSoftmax(dim=1)
)

funcao_erro = nn.CrossEntropyLoss(weight=weights)
otimizador = optim.AdamW(resnet50.parameters(), lr=0.0001)
scheduler = ReduceLROnPlateau(otimizador, mode='min', factor=0.1, patience=3)


def treinar_e_validar_cross_validation(modelo, metrica_erro, otimizador_sgd, scheduler, epocas=25, folds=5):
    kf = KFold(n_splits=folds, shuffle=True, random_state=42)
    fold_results = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(data['treino'])):
        print(f"\nIniciando Fold {fold + 1}/{folds}")

        train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
        val_sampler = torch.utils.data.SubsetRandomSampler(val_idx)
        
        data_loader_train_fold = DataLoader(data['treino'], batch_size=bs, sampler=train_sampler)
        data_loader_val_fold = DataLoader(data['treino'], batch_size=bs, sampler=val_sampler)

        melhor_acuracia = 0.0
        melhor_modelo = None

        for epoca in range(epocas):
            inicio_epoca = time.time()
            print(f"\nFold {fold + 1}, Época {epoca + 1}/{epocas}")

            modelo.train()
            erro_treino = 0.0
            acuracia_treino = 0.0

            for entradas, labels in tqdm(data_loader_train_fold, desc="Treinando"):
                entradas = entradas.to(device)
                labels = labels.to(device)

                otimizador_sgd.zero_grad()
                saidas = modelo(entradas)
                erro = metrica_erro(saidas, labels)
                erro.backward()
                otimizador_sgd.step()

                erro_treino += erro.item() * entradas.size(0)

            erro_medio_treino = erro_treino / len(train_idx)

            modelo.eval()
            erro_validacao = 0.0
            acuracia_validacao = 0.0

            with torch.no_grad():
                for entradas, labels in tqdm(data_loader_val_fold, desc="Validando"):
                    entradas = entradas.to(device)
                    labels = labels.to(device)
                    saidas = modelo(entradas)
                    erro = metrica_erro(saidas, labels)
                    erro_validacao += erro.item() * entradas.size(0)

                erro_medio_validacao = erro_validacao / len(val_idx)

                print(f"\nFold {fold + 1}, Época {epoca + 1}: Erro Treino {erro_medio_treino:.4f}, Erro Validação {erro_medio_validacao:.4f}")
                # Salvar o melhor modelo baseado na validação
                if erro_medio_validacao < melhor_acuracia or melhor_modelo is None:
                    melhor_acuracia = erro_medio_validacao
                    melhor_modelo = modelo.state_dict()
                    torch.save(melhor_modelo, f'./melhores_modelos/melhor_modelo_fold02{fold + 1}.pt')

            scheduler.step(erro_medio_validacao)

        fold_results.append(erro_medio_validacao)

    print(f"\nErro Médio Validação (Cross-Validation): {np.mean(fold_results):.4f}")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f'{device}\n')

numero_de_epocas = 20

resnet50 = resnet50.to(device)
treinar_e_validar_cross_validation(resnet50, funcao_erro, otimizador, scheduler, numero_de_epocas, folds=5)


{0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Neutral', 5: 'Sad', 6: 'Surprise'}


c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


cuda:0


Iniciando Fold 1/5

Fold 1, Época 1/20


Validando: 100%|██████████| 180/180 [00:22<00:00,  8.15it/s]



Fold 1, Época 1: Erro Treino 1.6557, Erro Validação 1.5847

Fold 1, Época 2/20


...


Fold 5, Época 18: Erro Treino 0.6859, Erro Validação 0.5901

Fold 5, Época 19/20


Validando: 100%|██████████| 180/180 [00:20<00:00,  8.94it/s]



Fold 5, Época 19: Erro Treino 0.6753, Erro Validação 0.5888

Fold 5, Época 20/20


Validando: 100%|██████████| 180/180 [00:20<00:00,  8.96it/s]


Fold 5, Época 20: Erro Treino 0.6649, Erro Validação 0.5938

Erro Médio Validação (Cross-Validation): 0.7628
